In [2]:
import geopandas as gpd
import pandas as pd
import os
import glob

# --- KONFIGURASI ---
INPUT_FOLDER = "raw_geojson"  # Folder tempat file terpisah disimpan
OUTPUT_FILE = "data/kecamatan_jakarta_full.geojson" # File hasil gabungan

def merge_geojson_files():
    # 1. Cari semua file .geojson di folder input
    file_list = glob.glob(os.path.join(INPUT_FOLDER, "*.geojson"))
    
    if not file_list:
        print(f"❌ Tidak ada file GeoJSON ditemukan di {INPUT_FOLDER}")
        return

    print(f"📂 Ditemukan {len(file_list)} file GeoJSON. Memulai penggabungan...")

    gdfs = []
    for file_path in file_list:
        try:
            # Membaca setiap file GeoJSON
            gdf = gpd.read_file(file_path)
            
            # Opsional: Tambahkan kolom asal file jika perlu untuk debugging
            # gdf['source_file'] = os.path.basename(file_path)
            
            gdfs.append(gdf)
            print(f"   ✅ Berhasil memuat: {os.path.basename(file_path)}")
        except Exception as e:
            print(f"   ❌ Gagal memuat {os.path.basename(file_path)}: {e}")

    if gdfs:
        # 2. Menggabungkan semua GeoDataFrame menjadi satu
        # ignore_index=True penting agar index direset
        merged_gdf = pd.concat(gdfs, ignore_index=True)

        # 3. Pastikan format CRS (Coordinate Reference System) konsisten (WGS84)
        if merged_gdf.crs is None:
             merged_gdf.set_crs(epsg=4326, inplace=True)
        else:
             merged_gdf = merged_gdf.to_crs(epsg=4326)

        # 4. Simpan ke file baru
        merged_gdf.to_file(OUTPUT_FILE, driver="GeoJSON")
        
        print("\n" + "="*50)
        print(f"🎉 SUKSES! File gabungan disimpan di: {OUTPUT_FILE}")
        print(f"📊 Total Fitur (Kecamatan/Kelurahan): {len(merged_gdf)}")
        print("="*50)
        
        # Tampilkan beberapa baris pertama untuk verifikasi
        print(merged_gdf.head())
    else:
        print("❌ Tidak ada data yang bisa digabungkan.")

if __name__ == "__main__":
    # Pastikan folder output ada
    output_dir = os.path.dirname(OUTPUT_FILE)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    merge_geojson_files()

📂 Ditemukan 6 file GeoJSON. Memulai penggabungan...
   ✅ Berhasil memuat: 31.01_kecamatan.geojson
   ✅ Berhasil memuat: 31.71_kecamatan.geojson
   ✅ Berhasil memuat: 31.72_kecamatan.geojson
   ✅ Berhasil memuat: 31.73_kecamatan.geojson
   ✅ Berhasil memuat: 31.74_kecamatan.geojson
   ✅ Berhasil memuat: 31.75_kecamatan.geojson

🎉 SUKSES! File gabungan disimpan di: data/kecamatan_jakarta_full.geojson
📊 Total Fitur (Kecamatan/Kelurahan): 44
  kd_propinsi kd_dati2 kd_kecamatan              nm_kecamatan  \
0          31       01          001    Kepulauan Seribu Utara   
1          31       01          002  Kepulauan Seribu Selatan   
2          31       71          001                    Gambir   
3          31       71          002               Sawah Besar   
4          31       71          003                 Kemayoran   

                                            geometry  
0  MULTIPOLYGON Z (((106.54303 -5.69973 0, 106.54...  
1  MULTIPOLYGON Z (((106.52136 -5.95971 0, 106.52...  
2 

In [3]:
import geopandas as gpd
g = gpd.read_file("data/kecamatan_jakarta_full.geojson")
print(g.columns)

Index(['kd_propinsi', 'kd_dati2', 'kd_kecamatan', 'nm_kecamatan', 'geometry'], dtype='object')


In [5]:
import geopandas as gpd
g = gpd.read_file("data/kecamatan_jakarta_full.geojson")
print(g['nm_kecamatan'].unique())


['Kepulauan Seribu Utara' 'Kepulauan Seribu Selatan' 'Gambir'
 'Sawah Besar' 'Kemayoran' 'Senen' 'Cempaka Putih' 'Menteng' 'Tanah Abang'
 'Johar Baru' 'Penjaringan' 'Tanjung Priok' 'Koja' 'Cilincing'
 'Pademangan' 'Kelapa Gading' 'Cengkareng' 'Grogol Petamburan'
 'Taman Sari' 'Tambora' 'Kebon Jeruk' 'Kalideres' 'Pal Merah' 'Kembangan'
 'Pasar Minggu' 'Kebayoran Lama' 'Cilandak' 'Kebayoran Baru' 'Pancoran'
 'Jagakarsa' 'Pesanggrahan' 'Tebet' 'Setiabudi' 'Mampang Prapatan'
 'Matraman' 'Pulogadung' 'Jatinegara' 'Kramatjati' 'Pasar Rebo' 'Cakung'
 'Duren Sawit' 'Makasar' 'Ciracas' 'Cipayung']


In [6]:
import geopandas as gpd

INPUT_FILE = "kecamatan_jakarta_full.geojson" # Ganti dengan file Anda
OUTPUT_FILE = "kecamatan_fixed.geojson"

# 1. Buka File
gdf = gpd.read_file(INPUT_FILE)

# 2. Cek kolom yang ada
print("Kolom sebelum:", gdf.columns)

# 3. Rename kolom
if 'nm_kecamatan' in gdf.columns:
    gdf = gdf.rename(columns={'nm_kecamatan': 'district'})
    print("✅ Berhasil mengubah 'nm_kecamatan' menjadi 'district'")

# 4. Simpan kembali
gdf.to_file(OUTPUT_FILE, driver="GeoJSON")
print("Kolom sesudah:", gdf.columns)

Kolom sebelum: Index(['kd_propinsi', 'kd_dati2', 'kd_kecamatan', 'nm_kecamatan', 'geometry'], dtype='object')
✅ Berhasil mengubah 'nm_kecamatan' menjadi 'district'
Kolom sesudah: Index(['kd_propinsi', 'kd_dati2', 'kd_kecamatan', 'district', 'geometry'], dtype='object')


In [10]:
import geopandas as gpd
g = gpd.read_file("kecamatan.geojson")
print(g['district'].unique())

['Kepulauan Seribu Utara' 'Kepulauan Seribu Selatan' 'Gambir'
 'Sawah Besar' 'Kemayoran' 'Senen' 'Cempaka Putih' 'Menteng' 'Tanah Abang'
 'Johar Baru' 'Penjaringan' 'Tanjung Priok' 'Koja' 'Cilincing'
 'Pademangan' 'Kelapa Gading' 'Cengkareng' 'Grogol Petamburan'
 'Taman Sari' 'Tambora' 'Kebon Jeruk' 'Kalideres' 'Pal Merah' 'Kembangan'
 'Pasar Minggu' 'Kebayoran Lama' 'Cilandak' 'Kebayoran Baru' 'Pancoran'
 'Jagakarsa' 'Pesanggrahan' 'Tebet' 'Setiabudi' 'Mampang Prapatan'
 'Matraman' 'Pulogadung' 'Jatinegara' 'Kramatjati' 'Pasar Rebo' 'Cakung'
 'Duren Sawit' 'Makasar' 'Ciracas' 'Cipayung']
